<a href="https://colab.research.google.com/github/jmcord/TabGAN/blob/master/TabGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

/usr/local/lib/python3.10/dist-packages/_ctgan/synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [1]:
# HIDE OUTPUT
CMD = "https://github.com/Diyago/GAN-for-tabular-data/blob/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan
!pip install table_evaluator

/bin/bash: line 1: https://github.com/Diyago/GAN-for-tabular-data/blob/master/requirements.txt: No such file or directory


In [2]:
try:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [3]:
#Load example dataset from manu different synthetic distributions
dataset = pd.read_csv('/content/drive/MyDrive/ctgan/credit_card.csv', sep=',', index_col=[0],decimal='.')
dataset.head()

,id,total_outflow_do,digitais_vinc,nunca_digital_hace4,total_outflow_do_hace2,appp,digitais_vinc_hace2,nunca_digital_hace7,pontuacao_triad,n_mov_voluntarios,...,idaily_banking_hace4,d_0_n_contactos_push_notification_hace_1y7,cd_v_total_hace2,nfamilias_vinc_hace4,d_0_vcomissoes_vact_hace_1y13,vcomissoes,appp_hace13,d_0_subtotal_domiciliacao_pag_hace_1y13,cd_n_parcial_hace4,target
1,475160.0,0.00,0,1,0.00,0,0,1,8,0,...,0,0.0,0.00,1,0.07674,97.92,0,0.000000,0,0
2,496928.0,57.06,1,0,65.82,0,1,0,12,19,...,1,0.0,678.39,3,0.00000,51.68,0,0.000000,14,0
3,480404.0,0.00,0,1,0.00,0,0,1,12,40,...,0,0.0,0.00,2,0.00000,0.00,0,0.000000,0,0
4,501053.0,901.07,0,1,250.26,0,0,1,13,38,...,1,0.0,2441.25,2,0.00000,46.46,0,0.316817,12,0
5,503549.0,11.75,1,0,5.51,0,1,0,12,18,...,1,0.0,1498.80,3,0.00000,42.67,0,0.000000,21,0


In [4]:
import tensorflow
from sklearn.model_selection import train_test_split

In [ ]:
#Train a previous NN to the '' variable to compare performance against generated dataset

# HIDE OUTPUT
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA', '?'])

COLS_USED = ['cylinders', 'displacement', 'horsepower', 'weight',
          'acceleration', 'year', 'origin','mpg']
COLS_TRAIN = ['cylinders', 'displacement', 'horsepower', 'weight',
          'acceleration', 'year', 'origin']

df = df[COLS_USED]

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())


# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("mpg", axis=1),
    df["mpg"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

In [ ]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

In [ ]:
#Train/test
import random
test_size = 0.2
num_test_samples = int(len(dataset) * test_size)
test_indices = random.sample(range(len(dataset)), num_test_samples)
data_test = dataset.iloc[test_indices,:]
data_train = dataset.iloc[test_indices,:]
len(dataset)

In [6]:
# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    dataset.drop("target", axis=1),
    dataset["target"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values


In [ ]:
# generate data
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test, )
new_train2, new_target2 = GANGenerator().generate_data_pipe(train, target, test, )


In [ ]:
# example with all params defined
new_train3, new_target3 = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(train, target,
                                          test, deep_copy=True, only_adversarial=False, use_adversarial=True)

In [7]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,
          df_x_test, deep_copy=True, only_adversarial=False, use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/62 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

IntCastingNaNError: ignored

In [8]:
#Save
gen_x.to_csv('/content/drive/MyDrive/ctgan/gen_x.csv', index = False)
gen_y.to_csv('/content/drive/MyDrive/ctgan/gen_y.csv', index = False)

NameError: ignored

In [ ]:
gen_x

In [ ]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))